In [ ]:
import scanpy as sc
from pyDeepInsight import ImageTransformer
import numpy as np
from PIL import Image
from scipy.sparse import csr_matrix
import pandas as pd
import anndata
import bbknn
from matplotlib import pyplot as plt
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import h5py
from sklearn.cluster import AgglomerativeClustering
from scipy.interpolate import interp1d
import bbknn

##读取不同格式的基因表达量矩阵

adata = sc.read_h5ad("/home/GaoBX/scDMLdata/bct_raw/bct_raw.h5ad")

adata = pd.read_csv('/home/GaoBX/work/counts.csv',index_col=0)

adata = sc.read_mtx("/home/GaoBX/work/expression_matrix.mtx")

adata_copy = sc.AnnData(adata)

##读取细胞类别文件，一般为“meatdata”文件
df = pd.read_csv("/home/GaoBX/work/metadata.csv",index_col=0)

true_labels = df['cluster']

##查看细胞原始类别个数
celltype_counts = pd.value_counts(true_labels)
print(celltype_counts)


##数据预处理部分
#sc.pp.filter_cells(adata_copy, min_genes=200) # 过滤一个细胞中表达少于200个基因的细胞样本
sc.pp.filter_genes(adata_copy, min_cells=3)# 过滤在少于3个细胞中表达的基因
sc.pp.normalize_total(adata_copy,target_sum=1e4)
sc.pp.log1p(adata_copy)
sc.pp.highly_variable_genes(adata_copy,n_top_genes=1000,subset=True)
sc.pp.scale(adata_copy)
sc.tl.pca(adata_copy)
sc.pp.neighbors(adata_copy)
sc.tl.umap(adata_copy)


##提取预处理过的表达量矩阵
matrix = adata_copy.X

##缺失值用0填充
matrix_filled = np.nan_to_num(matrix, nan=0.0)

##导出表达量矩阵
pd.DataFrame(matrix_filled).to_csv('/home/GaoBX/Study.xlsx', index=False)